In [1]:
import geopandas as gpd
from sqlalchemy import create_engine

In [3]:
! mc cp s3/projet-slums-detection/Stagios/stats_ilots.gpkg stats_ilots.gpkg

]11;?\

...ilots.gpkg: 43.32 MiB / 43.32 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 89.45 MiB/s 0s

In [ ]:
! pip install geopandas psycopg2-binary
! pip install geoalchemy2

In [2]:
user = 'cratt'
password = 'cratt'
host = '10.233.65.71'
port = '5432'
dbname = 'db_cratt'
schema = 'public'  # ou un autre schéma si nécessaire

connection_string = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'
engine = create_engine(connection_string)



In [18]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, Float, String, DateTime, select, func
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

from geoalchemy2 import Geometry
from geoalchemy2 import Geometry, WKTElement
import pandas as pd

In [4]:
connection_string = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'
engine = create_engine(connection_string)

gpkg_path = 'stats_ilots.gpkg'

gdf = gpd.read_file(gpkg_path)

gdf = gdf.to_crs(4326)
gdf.head(2)

,ident_ilot,aire_bati,aireplus,airemoins,airetotale,aire_bati_par_ilot,aire_plus_pct,aire_moins_pct,aire_totale_pct,area,geometry
0,976110325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9061.011359,"MULTIPOLYGON (((45.22039 -12.77447, 45.22018 -..."
1,976110411,6.75,76.25,66.5,9.75,17472.5,0.4364,0.380598,0.055802,15622.671586,"MULTIPOLYGON (((45.21049 -12.80063, 45.21077 -..."


In [5]:
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()

/tmp/ipykernel_47907/163760981.py:3: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [6]:
class GwWell(Base):
    __tablename__ = 'stat_ilot'
    ident_ilot = Column(String, primary_key=True)
    aire_bati = Column(Float)
    aireplus = Column(Float)
    airemoins = Column(Float)
    airetotale = Column(Float)
    aire_bati_par_ilot = Column(Float)
    aire_plus_pct =  Column(Float)
    aire_moins_pct =  Column(Float)
    aire_totale_pct =  Column(Float)
    area =  Column(Float)
    geom = Column(Geometry(geometry_type='MULTIPOLYGON', srid='4326'))

GwWell.__table__.create(engine)
GwWell.__table__

ProgrammingError: (psycopg2.errors.DuplicateTable) relation "stat_ilot" already exists

[SQL: 
CREATE TABLE stat_ilot (
	ident_ilot VARCHAR NOT NULL, 
	aire_bati FLOAT, 
	aireplus FLOAT, 
	airemoins FLOAT, 
	airetotale FLOAT, 
	aire_bati_par_ilot FLOAT, 
	aire_plus_pct FLOAT, 
	aire_moins_pct FLOAT, 
	aire_totale_pct FLOAT, 
	area FLOAT, 
	geom geometry(MULTIPOLYGON,4326), 
	PRIMARY KEY (ident_ilot)
)

]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [7]:
gdf.head(1)

,ident_ilot,aire_bati,aireplus,airemoins,airetotale,aire_bati_par_ilot,aire_plus_pct,aire_moins_pct,aire_totale_pct,area,geometry
0,976110325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9061.011359,"MULTIPOLYGON (((45.22039 -12.77447, 45.22018 -..."


In [16]:
for index, row in gdf.iterrows():
    well = GwWell(
        ident_ilot = row.ident_ilot,
        aire_bati = row.aire_bati,
        aireplus = row.aireplus,
        airemoins = row.airemoins,
        airetotale = row.airetotale,
        aire_bati_par_ilot = row.aire_bati_par_ilot,
        aire_plus_pct = row.aire_plus_pct,
        aire_moins_pct = row.aire_moins_pct,
        aire_totale_pct =row.aire_totale_pct,
        area = row.area,
        geom = WKTElement(row.geometry.wkt, srid=4326)
    )
    session.add(well)

# save changes on the session
session.commit()

/tmp/ipykernel_47907/3588937828.py:1: SAWarning: Session's state has been changed on a non-active transaction - this state will be discarded.
  session.rollback()


In [20]:
stmt = select(GwWell)
result = session.execute(stmt)

compteur  = 0
for row in result:
    compteur +=1
    print(row)
print(compteur)

(<__main__.GwWell object at 0x7f115e587050>,)
(<__main__.GwWell object at 0x7f115e57c0d0>,)
(<__main__.GwWell object at 0x7f115faa42d0>,)
(<__main__.GwWell object at 0x7f115e5836d0>,)
(<__main__.GwWell object at 0x7f115e583910>,)
(<__main__.GwWell object at 0x7f115e583d10>,)
(<__main__.GwWell object at 0x7f115e583e10>,)
(<__main__.GwWell object at 0x7f115e583c10>,)
(<__main__.GwWell object at 0x7f115e583e90>,)
(<__main__.GwWell object at 0x7f115e583b90>,)
(<__main__.GwWell object at 0x7f115e583850>,)
(<__main__.GwWell object at 0x7f115e583e50>,)
(<__main__.GwWell object at 0x7f11606fded0>,)
(<__main__.GwWell object at 0x7f115e5806d0>,)
(<__main__.GwWell object at 0x7f115e5830d0>,)
(<__main__.GwWell object at 0x7f115e580bd0>,)
(<__main__.GwWell object at 0x7f115e583050>,)
(<__main__.GwWell object at 0x7f115e54d810>,)
(<__main__.GwWell object at 0x7f115e54dc10>,)
(<__main__.GwWell object at 0x7f115e54fb50>,)
(<__main__.GwWell object at 0x7f115e54dd50>,)
(<__main__.GwWell object at 0x7f11